In [2]:
import pandas as pd
import numpy as np
import jieba as jb
from sklearn.model_selection import train_test_split

In [24]:
dict_people={'廖書峰':0,'何遠榮':1,'林妤臻':2,'莊念恩':3,'陳韋仁':4,'林柏安':5,'王佩琪':6,'吳珮如':7,'李柏諠':8}
dict_people_reverse={0:'廖書峰',1:'何遠榮',2:'林妤臻',3:'莊念恩',4:'陳韋仁',5:'林柏安',6:'王佩琪',7:'吳珮如',8:'李柏諠'}

# 匯入data

In [3]:
data=pd.read_excel('C:\\Users\\User\\OneDrive\\file\\category3.xlsx')
data.head()

,承辦人,關鍵字
0,王佩琪,請貴署王佩琪秘書提供有關合法爆竹煙火工廠發生火災意外事故或其
1,王佩琪,研議爆竹煙火輸入資格等相關事項之會議紀
2,王佩琪,為巡安堂貿易商行違反爆竹煙火管理條例
3,王佩琪,強化專業爆竹煙火管理研商會議
4,王佩琪,為彙復監察院有關非管制品之信號彈


# data前處理

In [4]:
#####將表中的關鍵字全部切成詞並統整轉成一個字典(word_index)#####
def make_dict(dataframe):
    global word_list,word_index
    word_list=[]  #還沒去標點符號的list
    word_list2=[] #去完標點符號的list
    trash_word=['。','，',' ','「','」','(',')',':','、']
    for i in range(len(dataframe['關鍵字'])):
        word_cut=','.join(jb.cut(dataframe['關鍵字'][i]))#切成字串
        word_cut=word_cut.split(sep=',')#將字串轉成list
        word_list.extend(word_cut)#將list彙整
        word_list=list(set(word_list))#去除重複
    for i in word_list:#去除贅字
        if i in trash_word:
            pass
        else:
            word_list2.append(i)
    #將list轉成字典
    word_index=pd.Series(index=word_list2,data=range(len(word_list2)))

In [5]:
#####將表中關鍵字分別轉成向量#####
def add_matrix(dataframe):
    trash_word=['。','，',' ','「','」','(',')',':','、']

###將人編號###
    dataframe['承辦人ID']=dataframe['承辦人'].map(dict_people)
    
###將關鍵字轉成向量###
    for i in range(len(dataframe['關鍵字'])):
        #切成字串
        dataframe['matrix'][i]=','.join(jb.cut(dataframe['關鍵字'][i]))
        dataframe['matrix'][i]=dataframe['matrix'][i].split(sep=',')
        #去除贅字
        list_matrix=[]
        for j in dataframe['matrix'][i]:
            if j in trash_word:
                pass
            else:
                list_matrix.append(j)
        dataframe['matrix'][i]=list_matrix
        #轉成數字向量
        dataframe['matrix'][i]=list(pd.Series(dataframe['matrix'][i]).map(word_index))

In [26]:
make_dict(data)

In [27]:
add_matrix(data)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: Sett

In [62]:
data['承辦人ID'].unique()

array([6., 1., 7., 8., 3., 2., 5., 4., 0.])

In [63]:
#將matrix補成20陣列
A=tf.keras.preprocessing.sequence.pad_sequences(data['matrix'],20)
a=np.array(data['承辦人ID'])
A.shape,a.shape

((269, 20), (269,))

In [72]:
#切割train\test數據集
X_train, X_test, y_train, y_test=train_test_split(A,a,test_size=0.2,random_state=0)
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((215, 20), (54, 20), (215,), (54,))

In [73]:
word_index.shape

(626,)

# 兜model

In [77]:
jmda_model=tf.keras.Sequential()
jmda_model.add(tf.keras.layers.Embedding(input_dim=626,output_dim=512,input_shape=(20,)))
jmda_model.add(tf.keras.layers.LSTM(512,activation='tanh'))
jmda_model.add(tf.keras.layers.Dropout(0.2))
jmda_model.add(tf.keras.layers.Dense(32,activation='relu'))
jmda_model.add(tf.keras.layers.Dropout(0.2))
jmda_model.add(tf.keras.layers.Dense(32,activation='relu'))
jmda_model.add(tf.keras.layers.Dense(9,activation='softmax'))
jmda_model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [78]:
jmda_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 512)           320512    
_________________________________________________________________
lstm_6 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                16416     
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)               

In [80]:
jmda_model.fit(X_train,y_train,epochs=300,validation_data=(X_test,y_test))

Train on 215 samples, validate on 54 samples
Epoch 1/300
215/215 [==============================] - 1s 7ms/sample - loss: 0.7914 - accuracy: 0.6651 - val_loss: 2.6649 - val_accuracy: 0.3704
Epoch 2/300
215/215 [==============================] - 1s 7ms/sample - loss: 0.7747 - accuracy: 0.6744 - val_loss: 2.5561 - val_accuracy: 0.4444
Epoch 3/300
215/215 [==============================] - 1s 7ms/sample - loss: 0.7610 - accuracy: 0.7163 - val_loss: 2.3023 - val_accuracy: 0.4630
Epoch 4/300
215/215 [==============================] - 1s 7ms/sample - loss: 0.7326 - accuracy: 0.7070 - val_loss: 2.2031 - val_accuracy: 0.4630
Epoch 5/300
215/215 [==============================] - 1s 7ms/sample - loss: 0.7464 - accuracy: 0.7023 - val_loss: 2.5229 - val_accuracy: 0.4444
Epoch 6/300
215/215 [==============================] - 1s 7ms/sample - loss: 0.7106 - accuracy: 0.6930 - val_loss: 2.3346 - val_accuracy: 0.4259
Epoch 7/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.6325 -

Epoch 57/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.2304 - accuracy: 0.9256 - val_loss: 3.5434 - val_accuracy: 0.4259
Epoch 58/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.2631 - accuracy: 0.9116 - val_loss: 3.6328 - val_accuracy: 0.4444
Epoch 59/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.2448 - accuracy: 0.9116 - val_loss: 3.7345 - val_accuracy: 0.4815
Epoch 60/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.2280 - accuracy: 0.9116 - val_loss: 4.2657 - val_accuracy: 0.4630
Epoch 61/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.2942 - accuracy: 0.8977 - val_loss: 4.4941 - val_accuracy: 0.4259
Epoch 62/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.2377 - accuracy: 0.9116 - val_loss: 3.9823 - val_accuracy: 0.4815
Epoch 63/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.2784 - accuracy: 0.9302 - val_loss: 4.7819 -

215/215 [==============================] - 2s 8ms/sample - loss: 0.1513 - accuracy: 0.9581 - val_loss: 3.7467 - val_accuracy: 0.3889
Epoch 114/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.0785 - accuracy: 0.9767 - val_loss: 4.0418 - val_accuracy: 0.4444
Epoch 115/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.1176 - accuracy: 0.9674 - val_loss: 3.8206 - val_accuracy: 0.3519
Epoch 116/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.2502 - accuracy: 0.9674 - val_loss: 4.9328 - val_accuracy: 0.4074
Epoch 117/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.1366 - accuracy: 0.9721 - val_loss: 4.1987 - val_accuracy: 0.3889
Epoch 118/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.1627 - accuracy: 0.9488 - val_loss: 3.9371 - val_accuracy: 0.3519
Epoch 119/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.1533 - accuracy: 0.9674 - val_loss: 3.7217 - val_ac

Epoch 169/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.1334 - accuracy: 0.9721 - val_loss: 4.0627 - val_accuracy: 0.4074
Epoch 170/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.0861 - accuracy: 0.9721 - val_loss: 4.4319 - val_accuracy: 0.3889
Epoch 171/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.0871 - accuracy: 0.9581 - val_loss: 4.8606 - val_accuracy: 0.4630
Epoch 172/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.1208 - accuracy: 0.9860 - val_loss: 5.1670 - val_accuracy: 0.4259
Epoch 173/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.0883 - accuracy: 0.9721 - val_loss: 4.9600 - val_accuracy: 0.4259
Epoch 174/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.1054 - accuracy: 0.9674 - val_loss: 4.4693 - val_accuracy: 0.4259
Epoch 175/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.0649 - accuracy: 0.9860 - val_loss: 5

Epoch 225/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.0667 - accuracy: 0.9814 - val_loss: 5.1843 - val_accuracy: 0.3519
Epoch 226/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.0647 - accuracy: 0.9860 - val_loss: 5.2208 - val_accuracy: 0.3333
Epoch 227/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.0808 - accuracy: 0.9721 - val_loss: 5.5238 - val_accuracy: 0.3333
Epoch 228/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.0729 - accuracy: 0.9814 - val_loss: 5.5433 - val_accuracy: 0.3333
Epoch 229/300
215/215 [==============================] - 2s 7ms/sample - loss: 0.0376 - accuracy: 0.9907 - val_loss: 5.2849 - val_accuracy: 0.3519
Epoch 230/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.0798 - accuracy: 0.9767 - val_loss: 7.1381 - val_accuracy: 0.3704
Epoch 231/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.0356 - accuracy: 0.9814 - val_loss: 8

Epoch 281/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.0644 - accuracy: 0.9814 - val_loss: 5.6317 - val_accuracy: 0.4259
Epoch 282/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.0553 - accuracy: 0.9767 - val_loss: 5.7501 - val_accuracy: 0.4259
Epoch 283/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.0625 - accuracy: 0.9814 - val_loss: 5.9755 - val_accuracy: 0.4259
Epoch 284/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.0726 - accuracy: 0.9814 - val_loss: 5.9035 - val_accuracy: 0.4074
Epoch 285/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.0932 - accuracy: 0.9860 - val_loss: 6.4841 - val_accuracy: 0.3889
Epoch 286/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.2823 - accuracy: 0.9767 - val_loss: 5.4201 - val_accuracy: 0.3889
Epoch 287/300
215/215 [==============================] - 2s 8ms/sample - loss: 0.0576 - accuracy: 0.9767 - val_loss: 5

# 預測

In [83]:
def judge(sequence):
    #將字串切割並轉成vector
    trash_word=['。','，',' ','「','」','(',')',':','、']
    sequence=','.join(jb.cut(sequence))
    sequence=sequence.split(sep=',')
    sequence2=[]
    for i in sequence:
        if i in trash_word:
            pass
        else:
            sequence2.append(i)
    sequence3=[]
    for i in sequence2:
        if i in word_index:
            sequence3.append(word_index[i])
    #將vector長度調整讓model_jmda可以吃
    sequence3=tf.keras.preprocessing.sequence.pad_sequences([sequence3],maxlen=20)
    #predict data
    answer_matrix=jmda_model.predict(sequence3)
    #將predict出來的格式轉成承辦人名字
    answer=dict_people_reverse[np.argmax(answer_matrix)]
    print('承辦人是:{}'.format(answer))

In [87]:
x=''
judge(x)

承辦人是:王佩琪
